<a href="https://colab.research.google.com/github/SianC7/LAIDS/blob/main/AE_MLP_Experiment_1_Resource_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Training & Test Data

In [1]:
import pandas as pd
import os
import numpy as np

# --- Data Collection ---

# Set pandas display options for wide output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

save_path = "/content/drive/MyDrive/Colab Notebooks/Honours Project/Sian's Models/AE-MLP files"

# Training sets
X_train = pd.read_csv(os.path.join(save_path, "AE_X_train.csv")).to_numpy()
y_train = pd.read_csv(os.path.join(save_path, "AE_y_train.csv")).to_numpy().ravel()

# Validation sets
X_val = pd.read_csv(os.path.join(save_path, "AE_X_val.csv")).to_numpy()
y_val = pd.read_csv(os.path.join(save_path, "AE_y_val.csv")).to_numpy().ravel()

# Classifier training sets
X_classifier_train = pd.read_csv(os.path.join(save_path, "adasyn_classifier_X_train.csv")).to_numpy() # MLP training data augumented using adasyn
y_classifier_train = pd.read_csv(os.path.join(save_path, "adasyn_classifier_y_train.csv")).to_numpy().ravel()

# Classifier validation sets
X_classifier_val = pd.read_csv(os.path.join(save_path, "classifier_X_val.csv")).to_numpy()
y_classifier_val = pd.read_csv(os.path.join(save_path, "classifier_y_val.csv")).to_numpy().ravel()

# Test sets
X_test = pd.read_csv(os.path.join(save_path, "X_test.csv")).to_numpy()
y_test = pd.read_csv(os.path.join(save_path, "y_test.csv")).to_numpy().ravel()


# Print class distribution for each set
print("\nAE Training class distribution:")
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

print("\nAE Validation class distribution:")
unique, counts = np.unique(y_val, return_counts=True)
print(dict(zip(unique, counts)))

print("\nClassfier Training class distribution:")
unique, counts = np.unique(y_classifier_train, return_counts=True)
print(dict(zip(unique, counts)))

print("\nClassifier Validation class distribution:")
unique, counts = np.unique(y_classifier_val, return_counts=True)
print(dict(zip(unique, counts)))

print("\nTest class distribution:")
unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))


AE Training class distribution:
{np.int64(0): np.int64(1357596)}

AE Validation class distribution:
{np.int64(0): np.int64(150844)}

Classfier Training class distribution:
{np.int64(0): np.int64(320544), np.int64(1): np.int64(286765), np.int64(2): np.int64(288505), np.int64(3): np.int64(288493), np.int64(4): np.int64(288577), np.int64(5): np.int64(288455), np.int64(6): np.int64(288550)}

Classifier Validation class distribution:
{np.int64(0): np.int64(56567), np.int64(1): np.int64(10883), np.int64(2): np.int64(257), np.int64(3): np.int64(1098), np.int64(4): np.int64(15362), np.int64(5): np.int64(234), np.int64(6): np.int64(23250)}

Test class distribution:
{np.int64(0): np.int64(209506), np.int64(1): np.int64(18139), np.int64(2): np.int64(429), np.int64(3): np.int64(1830), np.int64(4): np.int64(25603), np.int64(5): np.int64(389), np.int64(6): np.int64(38749)}


Encode the labels

In [2]:
# --- Label Encoding ---

# Create a mapping from attack type to integer label
attack_type_map = {'Normal Traffic': 0, 'Port Scanning': 1, 'Web Attacks': 2, 'Brute Force': 3, 'DDoS': 4, 'Bots': 5, 'DoS': 6} # Use the specified mapping


Make sure datasets are numpy arrays for future operations

In [3]:
# List all datasets
datasets = {
    "X_train": X_train,
    "y_train": y_train,
    "X_val": X_val,
    "y_val": y_val,
    "X_classifier_train": X_classifier_train,
    "y_classifier_train": y_classifier_train,
    "X_classifier_val": X_classifier_val,
    "y_classifier_val": y_classifier_val,
    "X_test": X_test,
    "y_test": y_test
}

# Ensure all datasets are NumPy arrays
for name, data in datasets.items():
    if hasattr(data, "to_numpy"):
        datasets[name] = data.to_numpy().ravel() if "y_" in name else data.to_numpy()

# Update variables
X_train = datasets["X_train"]
y_train = datasets["y_train"]
X_val = datasets["X_val"]
y_val = datasets["y_val"]
X_classifier_train = datasets["X_classifier_train"]
y_classifier_train = datasets["y_classifier_train"]
X_classifier_val = datasets["X_classifier_val"]
y_classifier_val = datasets["y_classifier_val"]
X_test = datasets["X_test"]
y_test = datasets["y_test"]

#Set OS Constraints

In [4]:
import os
os.environ['TF_NUM_INTRAOP_THREADS'] = '4'
os.environ['TF_NUM_INTEROP_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '4'
os.environ['MKL_NUM_THREADS'] = '4'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


#Load in Models

In [5]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# -------------------------
# --- Load Keras Models ---
# -------------------------
ae_model_path = '/content/drive/MyDrive/Colab Notebooks/Honours Project/Sian\'s Models/AE-MLP files/Best_AE.keras'
mlp_model_path = '/content/drive/MyDrive/Colab Notebooks/Honours Project/Sian\'s Models/AE-MLP files/Best_MLP.keras'

ae_model = load_model(ae_model_path)
mlp_model = load_model(mlp_model_path)

ae_model_name_prefix = '/content/drive/MyDrive/Colab Notebooks/Honours Project/Sian\'s Models/AE-MLP files/AE'
mlp_model_name_prefix = '/content/drive/MyDrive/Colab Notebooks/Honours Project/Sian\'s Models/AE-MLP files/MLP'

# -------------------------
# --- Load Per-Feature Thresholds ---
# -------------------------
per_feature_thresholds_path = "/content/drive/MyDrive/Colab Notebooks/Honours Project/Sian's Models/AE-MLP files/ae_per_feature_thresholds.npy"
per_feature_thresholds = np.load(per_feature_thresholds_path)

# -------------------------
# --- Load TFLite Models ---
# -------------------------
def load_tflite_model(path):
    """Load TFLite model as bytes."""
    with open(path, 'rb') as f:
        return f.read()

# Base paths for saved TFLite models
ae_model_base = ae_model_name_prefix
mlp_model_base = mlp_model_name_prefix

# List of TFLite models (AE → MLP)
tflite_models = [
    {
        "ae_model": load_tflite_model(ae_model_base + '_float32.tflite'),
        "ae_name": "AE Float32 Model",
        "ae_filename": "_float32.tflite",
        "mlp_model": load_tflite_model(mlp_model_base + '_float32.tflite'),
        "mlp_name": "MLP Float32 Model",
        "mlp_filename": "_float32.tflite"
    },
    {
        "ae_model": load_tflite_model(ae_model_base + '_fp16_weights.tflite'),
        "ae_name": "AE Float16 Weights-Only Model",
        "ae_filename": "_fp16_weights.tflite",
        "mlp_model": load_tflite_model(mlp_model_base + '_fp16_weights.tflite'),
        "mlp_name": "MLP Float16 Weights-Only Model",
        "mlp_filename": "_fp16_weights.tflite"
    },
    {
        "ae_model": load_tflite_model(ae_model_base + '_int8_weights.tflite'),
        "ae_name": "AE Int8 Weights-Only Model",
        "ae_filename": "_int8_weights.tflite",
        "mlp_model": load_tflite_model(mlp_model_base + '_int8_weights.tflite'),
        "mlp_name": "MLP Int8 Weights-Only Model",
        "mlp_filename": "_int8_weights.tflite"
    }
]

print("All Keras and TFLite models loaded successfully.")


All Keras and TFLite models loaded successfully.


# Resource Evaluation of Dynamic Range Quantization (Weight-only quantisation)

Imports

In [6]:
# --- Imports ---
import os
import psutil
import time
import resource
import tensorflow as tf

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# --- Imports ---
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
import os

Set resource constraints

In [7]:
# Google Colab Resource Configuration


def configure_resources_for_colab():
    print("🔧 Configuring resources for Google Colab...")

    # 1. CPU Core Limitation (4 cores max)
    os.environ['OMP_NUM_THREADS'] = '4'
    os.environ['MKL_NUM_THREADS'] = '4'
    os.environ['NUMEXPR_NUM_THREADS'] = '4'
    os.environ['OPENBLAS_NUM_THREADS'] = '4'

    # 2. Set CPU affinity (works in Colab Linux environment)
    try:
        available_cores = min(4, os.cpu_count())
        os.sched_setaffinity(0, list(range(available_cores)))
        print(f"✅ CPU affinity set to {available_cores} cores")
    except Exception as e:
        print(f"⚠️ CPU affinity warning: {e}")

    # 3. Memory limitation (4GB)
    try:
        memory_limit = 4 * 1024 * 1024 * 1024  # 4GB in bytes
        resource.setrlimit(resource.RLIMIT_AS, (memory_limit, memory_limit))
        print(f"✅ Memory limit set to 4GB")
    except Exception as e:
        print(f"⚠️ Memory limit warning: {e}")

    # 4. CPU frequency monitoring (informational in Colab)
    try:
        cpu_info = psutil.cpu_freq()
        if cpu_info:
            print(f"ℹ️  CPU frequency: {cpu_info.current:.0f}MHz")
        else:
            print(f"ℹ️  CPU frequency info not available")
    except:
        print(f"ℹ️  Running on Google Colab - CPU frequency managed by platform")

    # 5. Process priority adjustment
    try:
        os.nice(5)  # Lower priority (0 is normal, positive is lower priority)
        print(f"✅ Process priority lowered")
    except Exception as e:
        print(f"⚠️ Priority adjustment warning: {e}")

    print("✅ Resource configuration complete for Colab environment")

# Call the function
configure_resources_for_colab()

# TensorFlow configuration (enhanced for Colab)
def configure_tensorflow_colab():
    try:
        # Try to limit CPU threads (only works before TF initialization)
        tf.config.threading.set_intra_op_parallelism_threads(4)
        tf.config.threading.set_inter_op_parallelism_threads(1)
        print("✅ TensorFlow CPU threads configured")
    except RuntimeError as e:
        print(f"⚠️ TensorFlow threading config: {str(e)}")
        print("ℹ️  TensorFlow was already initialized - CPU thread limits may not be applied")

    # GPU configuration (Colab often has GPU available)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Enable memory growth
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU memory growth enabled")

            # Optional: Set GPU memory limit (may fail if already configured)
            try:
                tf.config.experimental.set_virtual_device_configuration(
                    gpus[0],
                    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # 4GB GPU limit
                )
                print(f"✅ GPU memory limit set to 4GB")
            except RuntimeError as gpu_limit_error:
                print(f"ℹ️  GPU memory limit not set: {str(gpu_limit_error)}")

        except RuntimeError as e:
            print(f"⚠️ GPU configuration: {e}")
    else:
        print(f"ℹ️  No GPU detected - using CPU only")

    # Reduce TensorFlow verbosity
    tf.get_logger().setLevel('WARNING')
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    print("✅ TensorFlow configuration complete")

configure_tensorflow_colab()

# Resource monitoring for Colab
def monitor_colab_resources():
    try:
        # Memory usage
        mem = psutil.virtual_memory()
        print(f"📊 Memory: {mem.used / (1024**3):.1f}GB used / {mem.total / (1024**3):.1f}GB total ({mem.percent:.1f}%)")

        # CPU usage
        cpu_percent = psutil.cpu_percent(interval=1)
        print(f"📊 CPU usage: {cpu_percent:.1f}%")

        # Disk space
        disk = psutil.disk_usage('/')
        print(f"📊 Disk: {disk.used / (1024**3):.1f}GB used / {disk.total/(1024**3):.1f}GB total ({disk.percent:.1f}%)")

    except Exception as e:
        print(f"⚠️ Monitoring error: {e}")

monitor_colab_resources()

🔧 Configuring resources for Google Colab...
✅ CPU affinity set to 2 cores
✅ Memory limit set to 4GB
ℹ️  CPU frequency: 2000MHz
✅ Process priority lowered
✅ Resource configuration complete for Colab environment
⚠️ TensorFlow threading config: Intra op parallelism cannot be modified after initialization.
ℹ️  TensorFlow was already initialized - CPU thread limits may not be applied
⚠️ GPU configuration: Physical devices cannot be modified after being initialized
✅ TensorFlow configuration complete
📊 Memory: 3.0GB used / 12.7GB total (26.0%)
📊 CPU usage: 2.0%
📊 Disk: 41.7GB used / 112.6GB total (37.0%)


#Resource usage evaluation

In [8]:
import psutil
import time
import os
import numpy as np
import matplotlib.pyplot as plt

# Storage for usage tracking
global timestamps
global cpu_usages
global mem_usages
timestamps = []
cpu_usages = []
mem_usages = []

global process
process = psutil.Process(os.getpid())
global start_time_global

def record_usage():
    """Capture CPU%, Memory, and elapsed inference time."""
    now = time.time() - start_time_global
    cpu = process.cpu_percent(interval=None)  # non-blocking
    mem = process.memory_info().rss / (1024**2)  # MB
    timestamps.append(now)
    cpu_usages.append(cpu)
    mem_usages.append(mem)

# ==========================
# Evaluate each AE → MLP pair
# ==========================
for m in tflite_models:
    timestamps = []
    cpu_usages = []
    mem_usages = []

    ae_model_content = m["ae_model"]
    ae_name = m["ae_name"]
    ae_filename = m["ae_filename"]

    mlp_model_content = m["mlp_model"]
    mlp_name = m["mlp_name"]
    mlp_filename = m["mlp_filename"]

    print(f"\n--- Evaluating AE: {ae_name} → MLP: {mlp_name} ---")

    ae_file_path = os.path.join(os.path.dirname(ae_model_name_prefix),os.path.basename(ae_model_name_prefix) + ae_filename)
    mlp_file_path = os.path.join(os.path.dirname(mlp_model_name_prefix),os.path.basename(mlp_model_name_prefix) + mlp_filename)
    time.sleep(5)

    # --- Get Models ready ---
    # Autoencoder
    ae_interpreter = tf.lite.Interpreter(model_content=ae_model_content)
    ae_interpreter.allocate_tensors()
    ae_input_details = ae_interpreter.get_input_details()
    ae_output_details = ae_interpreter.get_output_details()

    #MLP
    mlp_interpreter = tf.lite.Interpreter(model_content=mlp_model_content)
    mlp_interpreter.allocate_tensors()
    mlp_input_details = mlp_interpreter.get_input_details()
    mlp_output_details = mlp_interpreter.get_output_details()

    # --- Start Autoencoder Resource Evaluation ---
    start_time = time.time()
    start_time_global = start_time

    record_usage()
    ae_y_pred_probs = []
    record_usage()
    for i in range(1):
        input_data = np.expand_dims(X_test[i], axis=0).astype(ae_input_details[0]['dtype'])
        ae_interpreter.set_tensor(ae_input_details[0]['index'], input_data)
        ae_interpreter.invoke()
        record_usage()
        ae_output_data = ae_interpreter.get_tensor(ae_output_details[0]['index'])
        ae_y_pred_probs.append(ae_output_data[0])
        record_usage()

    ae_y_pred_probs = np.array(ae_y_pred_probs)
    test_reconstruction_errors = np.abs(ae_y_pred_probs - X_test[i])
    ae_y_pred = (test_reconstruction_errors > per_feature_thresholds).any(axis=1).astype(int)
    malicious_pred_indices = np.flatnonzero(ae_y_pred)
    record_usage()

    # --- Start MLP Resource Evaluation ---
    if len(malicious_pred_indices) > 0:
        mlp_X_test = X_test[malicious_pred_indices]
        mlp_y_pred_probs = []
        record_usage()
        for i in range(1):
            input_data = np.expand_dims(mlp_X_test[i], axis=0).astype(mlp_input_details[0]['dtype'])
            mlp_interpreter.set_tensor(mlp_input_details[0]['index'], input_data)
            mlp_interpreter.invoke()
            record_usage()
            mlp_output_data = mlp_interpreter.get_tensor(mlp_output_details[0]['index'])
            mlp_y_pred_probs.append(mlp_output_data[0])
            record_usage()

        mlp_y_pred_probs = np.array(mlp_y_pred_probs)
        mlp_y_pred = np.argmax(mlp_y_pred_probs, axis=1)
        record_usage()

    end_time = time.time()

    # --- Final Measurement ---

    total_storage_mb = (os.path.getsize(ae_file_path) + os.path.getsize(mlp_file_path)) / (1024**2)
    inference = end_time - start_time
    avg_cpu = np.mean(cpu_usages)
    avg_mem = np.mean(mem_usages)
    print(f"\n--- Resource Usage for {ae_name} → {mlp_name} ---")
    print(f"Total storage size: {total_storage_mb} MB")
    print(f"Inference time: {inference:.4f} sec")
    print(f"Average CPU usage: {avg_cpu}%")
    print(f"Average memory usage: {avg_mem} MB")

    # ==========================
    # --- Plot CPU & RAM vs Time
    # ==========================
     # --- Save CPU & RAM Plots ---
    # fig_filename = f"{ae_name}_to_{mlp_name}_usage.png".replace(" ", "_")
    # plt.figure(figsize=(12,6))
    # plt.subplot(2,1,1)
    # plt.plot(timestamps, cpu_usages, marker='o', label="CPU Usage (%)")
    # plt.ylabel("CPU %")
    # plt.legend()
    # plt.subplot(2,1,2)
    # plt.plot(timestamps, mem_usages, marker='o', color="orange", label="RAM (MB)")
    # plt.xlabel("Inference Time (s)")
    # plt.ylabel("Memory (MB)")
    # plt.legend()
    # plt.suptitle(f"CPU & Memory Usage: {ae_name} → {mlp_name}")
    # plt.savefig(fig_filename, dpi=150, bbox_inches='tight')
    # plt.close()




--- Evaluating AE: AE Float32 Model → MLP: MLP Float32 Model ---


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)



--- Resource Usage for AE Float32 Model → MLP Float32 Model ---
Total storage size: 0.07562637329101562 MB
Inference time: 0.0012 sec
Average CPU usage: 0.0%
Average memory usage: 2696.6796875 MB

--- Evaluating AE: AE Float16 Weights-Only Model → MLP: MLP Float16 Weights-Only Model ---

--- Resource Usage for AE Float16 Weights-Only Model → MLP Float16 Weights-Only Model ---
Total storage size: 0.04409027099609375 MB
Inference time: 0.0006 sec
Average CPU usage: 0.04%
Average memory usage: 2696.6796875 MB

--- Evaluating AE: AE Int8 Weights-Only Model → MLP: MLP Int8 Weights-Only Model ---

--- Resource Usage for AE Int8 Weights-Only Model → MLP Int8 Weights-Only Model ---
Total storage size: 0.03321075439453125 MB
Inference time: 0.0009 sec
Average CPU usage: 0.0%
Average memory usage: 2696.6796875 MB


#Full integer 8 Quantisation of the model

In [9]:
# import numpy as np

# def ae_representative_dataset_gen():
#     # Randomly select 1000 indices
#     total_samples = len(X_train)
#     selected_indices = np.random.choice(total_samples, 1000, replace=False)

#     # Yield samples with shape suitable for a fully connected autoencoder: (1, feature_dim)
#     for i in selected_indices:
#         yield [X_train[i].astype(np.float32).reshape(1, -1)]


In [10]:
# import numpy as np

# def mlp_representative_dataset_gen():
#     # Get class distribution
#     class_types, class_counts = np.unique(y_classifier_train, return_counts=True)
#     min_samples_per_class = min(class_counts)

#     class_indices = [np.where(y_classifier_train == c)[0] for c in class_types]

#     # Pick equal number of samples per class (e.g., 100 per class)
#     selected_indices = []
#     for indices in class_indices:
#         selected_indices.extend(np.random.choice(indices, 100, replace=False))

#     np.random.shuffle(selected_indices)
#     representative_indices = selected_indices

#     # Yield samples reshaped for MLP: (1, feature_dim)
#     for i in representative_indices:
#         yield [X_classifier_train[i].astype(np.float32).reshape(1, -1)]


Autoencoder

In [11]:
# # --- AE Full 8 Integer Model Quantization ---
# converter = tf.lite.TFLiteConverter.from_keras_model(ae_model)

# # Create an int8 quantized model (requires representative dataset)
# converter.optimizations = [tf.lite.Optimize.DEFAULT] # Apply default optimizations

# #converter.target_spec.supported_types = [tf.int8] # Specify target data type as int8

# converter.representative_dataset = ae_representative_dataset_gen # Provide the representative dataset and ensure input dtype is float32
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]# Specify OpsSet must specifically targets the 8-bit integer quantized versions of the TensorFlow Lite built-in operations.

# # Set the input and output types to int8 for inference
# converter.inference_input_type = tf.int8
# converter.inference_output_type = tf.int8
# print("# --- AE Full 8 Integer Model Quantization ---")
# ae_quantModel_int8 = converter.convert() # Convert the model


# # --- AE Perform 16x8 Full Integer Quantization ---
# # This converts weights to int8 and activations to int16.
# #converter = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = ae_representative_dataset_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]# Specify the experimental ops set for 16x8 quantization - int16 activations and int8 weights.
# converter.inference_input_type = tf.int16
# converter.inference_output_type = tf.int16

# print("# --- AE Perform 16x8 Full Integer Quantization ---")
# ae_quant_int16x8_model = converter.convert()# Convert the model


# # --- Save models ---
# os.makedirs('models', exist_ok=True)
# ae_int8_full_path = ae_model_name_prefix + '_int8_full.tflite' # Define path
# with open(ae_int8_full_path, 'wb') as f: # Added _full to filename
#     f.write(ae_quantModel_int8)
# print(f"AE Int8 Full Integer model saved to: {os.path.abspath(ae_int8_full_path)}") # Print path

# ae_int16x8_full_path = ae_model_name_prefix + '_int16x8_full.tflite' # Define path
# with open(ae_int16x8_full_path, 'wb') as f: # Added _full to filename
#     f.write(ae_quant_int16x8_model)
# print(f"AE Int16x8 Full Integer model saved to: {os.path.abspath(ae_int16x8_full_path)}") # Print path

MLP

In [12]:
# # --- MLP Full 8 Integer Model Quantization ---
# converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model) # Changed to mlp_model

# # Create an int8 quantized model (requires representative dataset)
# converter.optimizations = [tf.lite.Optimize.DEFAULT] # Apply default optimizations

# converter.representative_dataset = mlp_representative_dataset_gen # Provide the representative dataset and ensure input dtype is float32
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]# Specify OpsSet must specifically targets the 8-bit integer quantized versions of the TensorFlow Lite built-in operations.

# # Set the input and output types to int8 for inference
# converter.inference_input_type = tf.int8
# converter.inference_output_type = tf.int8
# print("# --- MLP Full 8 Integer Model Quantization ---") # Changed to MLP
# mlp_quantModel_int8 = converter.convert() # Changed to mlp_quantModel_int8


# # --- MLP Perform 16x8 Full Integer Quantization --- # Changed to MLP
# # This converts weights to int8 and activations to int16.
# #converter = tf.lite.TFLiteConverter.from_keras_model(cnn_model) # This comment might be outdated, leaving as is.
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = mlp_representative_dataset_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]# Specify the experimental ops set for 16x8 quantization - int16 activations and int8 weights.
# converter.inference_input_type = tf.int16
# converter.inference_output_type = tf.int16

# print("# --- MLP Perform 16x8 Full Integer Quantization ---") # Changed to MLP
# mlp_quant_int16x8_model = converter.convert()# Changed to mlp_quant_int16x8_model


# # --- Save models ---
# os.makedirs('models', exist_ok=True)
# mlp_int8_full_path = mlp_model_name_prefix + '_int8_full.tflite' # Changed to mlp_model_name_prefix and mlp_int8_full_path
# with open(mlp_int8_full_path, 'wb') as f:
#     f.write(mlp_quantModel_int8) # Changed to mlp_quantModel_int8
# print(f"MLP Int8 Full Integer model saved to: {os.path.abspath(mlp_int8_full_path)}") # Changed to MLP and mlp_int8_full_path

# mlp_int16x8_full_path = mlp_model_name_prefix + '_int16x8_full.tflite' # Changed to mlp_model_name_prefix and mlp_int16x8_full_path
# with open(mlp_int16x8_full_path, 'wb') as f:
#     f.write(mlp_quant_int16x8_model) # Changed to mlp_quant_int16x8_model
# print(f"MLP Int16x8 Full Integer model saved to: {os.path.abspath(mlp_int16x8_full_path)}") # Changed to MLP and mlp_int16x8_full_path

#Evaluation of full int quant models

In [13]:
# # Google Colab Resource Configuration


# def configure_resources_for_colab():
#     print("🔧 Configuring resources for Google Colab...")

#     # 1. CPU Core Limitation (4 cores max)
#     os.environ['OMP_NUM_THREADS'] = '4'
#     os.environ['MKL_NUM_THREADS'] = '4'
#     os.environ['NUMEXPR_NUM_THREADS'] = '4'
#     os.environ['OPENBLAS_NUM_THREADS'] = '4'

#     # 2. Set CPU affinity (works in Colab Linux environment)
#     try:
#         available_cores = min(4, os.cpu_count())
#         os.sched_setaffinity(0, list(range(available_cores)))
#         print(f"✅ CPU affinity set to {available_cores} cores")
#     except Exception as e:
#         print(f"⚠️ CPU affinity warning: {e}")

#     # 3. Memory limitation (4GB)
#     try:
#         memory_limit = 4 * 1024 * 1024 * 1024  # 4GB in bytes
#         resource.setrlimit(resource.RLIMIT_AS, (memory_limit, memory_limit))
#         print(f"✅ Memory limit set to 4GB")
#     except Exception as e:
#         print(f"⚠️ Memory limit warning: {e}")

#     # 4. CPU frequency monitoring (informational in Colab)
#     try:
#         cpu_info = psutil.cpu_freq()
#         if cpu_info:
#             print(f"ℹ️  CPU frequency: {cpu_info.current:.0f}MHz")
#         else:
#             print(f"ℹ️  CPU frequency info not available")
#     except:
#         print(f"ℹ️  Running on Google Colab - CPU frequency managed by platform")

#     # 5. Process priority adjustment
#     try:
#         os.nice(5)  # Lower priority (0 is normal, positive is lower priority)
#         print(f"✅ Process priority lowered")
#     except Exception as e:
#         print(f"⚠️ Priority adjustment warning: {e}")

#     print("✅ Resource configuration complete for Colab environment")

# # Call the function
# configure_resources_for_colab()

# # TensorFlow configuration (enhanced for Colab)
# def configure_tensorflow_colab():
#     try:
#         # Try to limit CPU threads (only works before TF initialization)
#         tf.config.threading.set_intra_op_parallelism_threads(4)
#         tf.config.threading.set_inter_op_parallelism_threads(1)
#         print("✅ TensorFlow CPU threads configured")
#     except RuntimeError as e:
#         print(f"⚠️ TensorFlow threading config: {str(e)}")
#         print("ℹ️  TensorFlow was already initialized - CPU thread limits may not be applied")

#     # GPU configuration (Colab often has GPU available)
#     gpus = tf.config.experimental.list_physical_devices('GPU')
#     if gpus:
#         try:
#             # Enable memory growth
#             for gpu in gpus:
#                 tf.config.experimental.set_memory_growth(gpu, True)
#             print(f"✅ GPU memory growth enabled")

#             # Optional: Set GPU memory limit (may fail if already configured)
#             try:
#                 tf.config.experimental.set_virtual_device_configuration(
#                     gpus[0],
#                     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # 4GB GPU limit
#                 )
#                 print(f"✅ GPU memory limit set to 4GB")
#             except RuntimeError as gpu_limit_error:
#                 print(f"ℹ️  GPU memory limit not set: {str(gpu_limit_error)}")

#         except RuntimeError as e:
#             print(f"⚠️ GPU configuration: {e}")
#     else:
#         print(f"ℹ️  No GPU detected - using CPU only")

#     # Reduce TensorFlow verbosity
#     tf.get_logger().setLevel('WARNING')
#     os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#     print("✅ TensorFlow configuration complete")

# configure_tensorflow_colab()

# # Resource monitoring for Colab
# def monitor_colab_resources():
#     try:
#         # Memory usage
#         mem = psutil.virtual_memory()
#         print(f"📊 Memory: {mem.used / (1024**3):.1f}GB used / {mem.total / (1024**3):.1f}GB total ({mem.percent:.1f}%)")

#         # CPU usage
#         cpu_percent = psutil.cpu_percent(interval=1)
#         print(f"📊 CPU usage: {cpu_percent:.1f}%")

#         # Disk space
#         disk = psutil.disk_usage('/')
#         print(f"📊 Disk: {disk.used / (1024**3):.1f}GB used / {disk.total/(1024**3):.1f}GB total ({disk.percent:.1f}%)")

#     except Exception as e:
#         print(f"⚠️ Monitoring error: {e}")

# monitor_colab_resources()

In [14]:
# # -- Imports ---
# import os
# import psutil
# import time
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# import tensorflow as tf

In [15]:
# # List of TFLite models (AE -> MLP)
# tflite_models = [
#     {
#         "ae_model": ae_quantModel_int8,
#         "ae_name": "AE Full Int8 Quantized Model",
#         "ae_filename": "_int8_full.tflite",
#         "mlp_model": mlp_quantModel_int8,
#         "mlp_name": "MLP Full Int8 Quantized Model",
#         "mlp_filename": "_int8_full.tflite"
#     },
#     # {
#     #     "ae_model": ae_quant_int16x8_model,
#     #     "ae_name": "AE Int16x8 Quantized Model",
#     #     "ae_filename": "_int16x8_full.tflite",
#     #     "mlp_model": mlp_quant_int16x8_model,
#     #     "mlp_name": "MLP Int16x8 Quantized Model",
#     #     "mlp_filename": "_int16x8_full.tflite"
#     # }
# ]

In [16]:
# # --- Evaluate all AE → MLP TFLite models ---
# for m in tflite_models:
#     # --- Unpack model info ---
#     ae_model_content = m["ae_model"]
#     ae_name = m["ae_name"]
#     ae_filename = m["ae_filename"]

#     mlp_model_content = m["mlp_model"]
#     mlp_name = m["mlp_name"]
#     mlp_filename = m["mlp_filename"]

#     print(f"\n--- Evaluating AE: {ae_name} → MLP: {mlp_name} ---")

#     # --- Construct full file paths ---
#     ae_file_path = os.path.join(os.path.dirname(ae_model_name_prefix),os.path.basename(ae_model_name_prefix) + ae_filename)
#     mlp_file_path = os.path.join(os.path.dirname(mlp_model_name_prefix),os.path.basename(mlp_model_name_prefix) + mlp_filename)

#     # --- Autoencoder ---
#     ae_interpreter = tf.lite.Interpreter(model_content=ae_model_content)
#     ae_interpreter.allocate_tensors()
#     ae_input_details = ae_interpreter.get_input_details()[0]
#     ae_output_details = ae_interpreter.get_output_details()[0]

#     in_scale,  in_zp  = ae_input_details['quantization']
#     out_scale, out_zp = ae_output_details['quantization']
#     print(f"AE Input Details: {ae_input_details}")
#     print(f"AE Output Details: {ae_output_details}")

#     # Start resource tracking
#     ae_process = psutil.Process(os.getpid())
#     ae_mem_before = ae_process.memory_info().rss / (1024*1024)
#     ae_cpu_before = psutil.cpu_percent(interval=None)
#     ae_start_time = time.time()

#    # --- Make new Thresholds for quantized model ---
#     val_reconstructions = []
#     # --- Get dequantized reconstruction for normal validation data ---
#     for i in range(len(X_val)):
#         x = np.expand_dims(X_val[i].astype(np.float32), axis=0)

#         # Quantize input if necessary
#         if ae_input_details['dtype'] == np.int8:
#             xq = np.round(x / in_scale + in_zp).astype(np.int8)
#         else:
#             xq = x.astype(ae_input_details['dtype'])

#         ae_interpreter.set_tensor(ae_input_details['index'], xq)
#         ae_interpreter.invoke()
#         yq = ae_interpreter.get_tensor(ae_output_details['index'])

#         # Dequantize output to FP32
#         if ae_output_details['dtype'] == np.int8:
#             y = (yq.astype(np.float32) - out_zp) * out_scale
#         else:
#             y = yq.astype(np.float32)

#         val_reconstructions.append(y[0])

#     val_reconstructions = np.array(val_reconstructions)

#     # --- Calculate per-feature reconstruction errors on normal data ---
#     val_reconstruction_errors = np.abs(val_reconstructions - X_val)

#     # --- Define the new threshold using a statistical method ---
#     mean_feature_errors = np.mean(val_reconstruction_errors, axis=0)
#     std_feature_errors = np.std(val_reconstruction_errors, axis=0)
#     new_per_feature_thresholds = mean_feature_errors + std_feature_errors #3 * std_feature_errors

#     print(f"New Per-Feature Thresholds: {new_per_feature_thresholds}")

#     # AE predictions (loop per sample)
#     # --- AE Test Predictions ---
#     ae_y_pred_probs = []
#     for i in range(len(X_test)):
#         x = np.expand_dims(X_test[i].astype(np.float32), axis=0)  # Correct: shape [1, num_features]
#         if ae_input_details['dtype'] == np.int8:
#             xq = np.round(x / in_scale + in_zp).astype(np.int8)
#         else:
#             xq = x.astype(ae_input_details['dtype'])

#         ae_interpreter.set_tensor(ae_input_details['index'], xq)
#         ae_interpreter.invoke()
#         yq = ae_interpreter.get_tensor(ae_output_details['index'])

#         if ae_output_details['dtype'] == np.int8:
#             y = (yq.astype(np.float32) - out_zp) * out_scale
#         else:
#             y = yq.astype(np.float32)

#         ae_y_pred_probs.append(y[0])

#     ae_y_pred_probs = np.array(ae_y_pred_probs)

#     # --- Anomaly Detection ---
#     test_reconstruction_errors = np.abs(ae_y_pred_probs - X_test)
#     #print(per_feature_thresholds)

#     # ae_y_pred = (test_reconstruction_errors > (per_feature_thresholds)).any(axis=1).astype(int)
#     ae_y_pred = (test_reconstruction_errors > (new_per_feature_thresholds)).any(axis=1).astype(int)
#     malicious_pred_indices = np.flatnonzero(ae_y_pred)

#     # End resource tracking
#     ae_end_time = time.time()
#     ae_mem_after = ae_process.memory_info().rss / (1024*1024)
#     ae_cpu_after = psutil.cpu_percent(interval=None)

#     # Calculate resource usage
#     ae_storage_size_mb = os.path.getsize(ae_file_path) / (1024*1024)
#     ae_memory_used_mb = ae_mem_after - ae_mem_before
#     ae_cpu_usage_percent = ae_cpu_after - ae_cpu_before
#     ae_inference_time_sec = ae_end_time - ae_start_time

#     # --- Confusion Matrix ---

#     y_test_binary = (y_test != 0).astype(int)# Convert y_test to binary: 0 = Normal, 1 = Malware

#     ae_cm = confusion_matrix(y_test_binary, ae_y_pred)
#     labels = ["Normal", "Malware"]

#     plt.figure(figsize=(6, 4))
#     sns.heatmap(ae_cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
#     plt.title(f"{ae_name} - Confusion Matrix (Anomaly Detection)")
#     plt.xlabel("Predicted")
#     plt.ylabel("True")
#     plt.show()

#     # --- Accuracy & Classification Report ---
#     print("Accuracy:", accuracy_score(y_test, ae_y_pred))
#     print("Classification Report:")
#     print(classification_report(y_test_binary, ae_y_pred, target_names=labels, labels=[0, 1]))


#     # --- Malware vs Benign Metrics ---
#     # Confusion matrix layout (for binary classification):
#     # [[TN, FP],
#     #  [FN, TP]]
#     tn, fp, fn, tp = ae_cm.ravel()

#     total_malware = tp + fn
#     total_benign = tn + fp

#     malware_identified = tp
#     benign_misclassified = fp

#     percentage_malware_identified = (malware_identified / total_malware * 100) if total_malware > 0 else 0
#     percentage_benign_misclassified = (benign_misclassified / total_benign * 100) if total_benign > 0 else 0

#     print(f"Malware Identified: {malware_identified}/{total_malware} ({percentage_malware_identified:.2f}%)")
#     print(f"Benign Misclassified: {benign_misclassified}/{total_benign} ({percentage_benign_misclassified:.2f}%)")
#     print(f"TP: {tp}\nTN: {tn}\nFP: {fp}\nFN: {fn}")

#     # Display Resource Usage
#     print(f"\n--- Resource Usage for {ae_name} ---")
#     print(f"Storage size: {ae_storage_size_mb:.2f} MB")
#     print(f"Memory used during inference: {ae_memory_used_mb:.2f} MB")
#     print(f"CPU usage change: {ae_cpu_usage_percent:.2f}%")
#     print(f"Inference time: {ae_inference_time_sec:.4f} sec")





#     # --- MLP --- #
#     # Loads the TFLite model and gets it ready to make predictions
#     mlp_interpreter = tf.lite.Interpreter(model_content=mlp_model_content) # Create interpreter object that will read and run the TFLite model
#     mlp_interpreter.allocate_tensors() # Make the interpreter allocate memory
#     mlp_input_details = mlp_interpreter.get_input_details()[0] # Get expected shape and data type of the data the model needs to evaluate (built-in method)
#     mlp_output_details = mlp_interpreter.get_output_details()[0] #Sshape and data type the model will need to produce the results in

#     mlp_in_scale, mlp_in_zp = mlp_input_details['quantization']
#     mlp_out_scale, mlp_out_zp = mlp_output_details['quantization']
#     print(f"MLP Input Details: {mlp_input_details}")
#     print(f"MLP Output Details: {mlp_output_details}")

#     # Start Resource Measurement
#     mlp_process = psutil.Process(os.getpid())
#     mlp_mem_before = mlp_process.memory_info().rss / (1024 * 1024)  # MB
#     mlp_cpu_before = psutil.cpu_percent(interval=None)
#     mlp_start_time = time.time()

#     # --- MLP Predictions ---
#     mlp_y_pred_probs = []
#     mlp_X_test = X_test[malicious_pred_indices]
#     mlp_y_test = y_test[malicious_pred_indices]

#     for i in range(len(mlp_X_test)):
#         x = np.expand_dims(mlp_X_test[i].astype(np.float32), axis=0)  # shape [1, num_features]
#         if mlp_input_details['dtype'] == np.int8:
#             xq = np.round(x / mlp_in_scale + mlp_in_zp).astype(np.int8)
#         else:
#             xq = x.astype(mlp_input_details['dtype'])

#         mlp_interpreter.set_tensor(mlp_input_details['index'], xq)
#         mlp_interpreter.invoke()
#         yq = mlp_interpreter.get_tensor(mlp_output_details['index'])

#         if mlp_output_details['dtype'] == np.int8:
#             y = (yq.astype(np.float32) - mlp_out_zp) * mlp_out_scale
#         else:
#             y = yq.astype(np.float32)

#         mlp_y_pred_probs.append(y[0])

#     mlp_y_pred_probs = np.array(mlp_y_pred_probs)
#     mlp_y_pred = np.argmax(mlp_y_pred_probs, axis=1)

#     # End Resource Measurement
#     mlp_end_time = time.time()
#     mlp_mem_after = mlp_process.memory_info().rss / (1024 * 1024)
#     mlp_cpu_after = psutil.cpu_percent(interval=None)

#     # Calculate Resource Measurement
#     mlp_storage_size_mb = os.path.getsize(mlp_file_path) / (1024 * 1024) # Use the correct file path
#     mlp_memory_used_mb = mlp_mem_after - mlp_mem_before
#     mlp_cpu_usage_percent = mlp_cpu_after - mlp_cpu_before
#     mlp_inference_time_sec = mlp_end_time - mlp_start_time

#     # --- Confusion Matrix ---
#     cm = confusion_matrix(mlp_y_test, mlp_y_pred)
#     reverse_attack_type_map = {v: k for k, v in attack_type_map.items()}
#     labels = [reverse_attack_type_map.get(i, f'Unknown {i}') for i in range(cm.shape[0])]

#     plt.figure(figsize=(8, 4)) # Create confusion matrix plot
#     sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
#     plt.title(f"{mlp_name} - Confusion Matrix (Classification)")
#     plt.xlabel("Predicted")
#     plt.ylabel("True")
#     plt.show()

#     # Print accuracy & classification report
#     print("Accuracy:", accuracy_score(mlp_y_test, mlp_y_pred))
#     print("Classification Report:")
#     print(classification_report(mlp_y_test, mlp_y_pred, target_names=labels))

#     # Calculate malware vs benign sample metrics
#     normal_label = attack_type_map.get('Normal Traffic', None)
#     if normal_label is not None:

#         malware_identified = np.sum(np.diag(cm)) - (cm[normal_label, normal_label] if normal_label in mlp_y_test else 0)
#         total_malware = np.sum(cm) - (np.sum(cm[normal_label, :]) if normal_label in mlp_y_test else 0)
#         percentage_malware_identified = (malware_identified / total_malware * 100) if total_malware > 0 else 0

#         benign_misclassified = (np.sum(cm[normal_label, :]) - cm[normal_label, normal_label]) if normal_label in mlp_y_test else 0
#         total_benign = np.sum(cm[normal_label, :]) if normal_label in mlp_y_test else 0
#         percentage_benign_misclassified = (benign_misclassified / total_benign * 100) if total_benign > 0 else 0

#         print(f"Malware Identified: {malware_identified}/{total_malware} ({percentage_malware_identified:.2f}%)")
#         print(f"Benign Misclassified: {benign_misclassified}/{total_benign} ({percentage_benign_misclassified:.2f}%)")
#         print(f"TP: {malware_identified}, TN: {(cm[normal_label, normal_label] if normal_label in mlp_y_test else 0)}, FP: {benign_misclassified}, FN: {total_malware - malware_identified}")


#     # Display Resource Usage
#     print(f"\n--- Resource Usage for {mlp_name} ---")
#     print(f"Storage size: {mlp_storage_size_mb:.2f} MB")
#     print(f"Memory used during inference: {mlp_memory_used_mb:.2f} MB")
#     print(f"CPU usage change: {mlp_cpu_usage_percent:.2f}%")
#     print(f"Inference time: {mlp_inference_time_sec:.4f} sec")

#     # Add a pause to allow CPU to reset
#     time.sleep(5) # Pause for 5 seconds